In [1]:
import ray

In [2]:
ray.init()

2021-03-03 18:04:39,193	INFO services.py:1174 -- View the Ray dashboard at http://127.0.0.1:8265


2021-03-03 18:04:39,198	WARNING services.py:1628 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=Xgb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 2gb.


{'node_ip_address': '172.31.0.17',
 'raylet_ip_address': '172.31.0.17',
 'redis_address': '172.31.0.17:6379',
 'object_store_address': '/tmp/ray/session_2021-03-03_18-04-38_593014_14/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-03-03_18-04-38_593014_14/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-03-03_18-04-38_593014_14',
 'metrics_export_port': 41999,
 'node_id': '64059ef4d674811eaba2fde5042fb28e5086acbb0b7672eaa835ca12'}

In [3]:
@ray.remote
class Counter(object):
    def __init__(self):
        self.n = 0
    
    def increment(self):
        self.n += 1
    
    def read(self):
        return self.n

In [4]:
counters = [Counter.remote() for i in range(4)]
[c.increment.remote() for c in counters]
futures = [c.read.remote() for c in counters]
ray.get(futures)

[1, 1, 1, 1]

In [5]:
from ray import tune

In [6]:
def objective(step, alpha, beta):
    return (0.1 + alpha * step / 100)**(-1) + beta * 0.1

In [7]:
def training_function(config):
    alpha, beta = config["alpha"], config["beta"]
    
    for step in range(10):
        intermediate_score = objective(step, alpha, beta)
        tune.report(mean_loss=intermediate_score)

In [8]:
analysis = tune.run(
    training_function,
    config={
        "alpha": tune.grid_search([0.001, 0.01, 0.1]),
        "beta": tune.choice([1, 2, 3])
    }
)

print("Best config ", analysis.get_best_config(
    metric="mean_loss", mode="min"
))

df = analysis.results_df

2021-03-03 18:04:42,478	WARNING function_runner.py:541 -- Function checkpointing is disabled. This may result in unexpected behavior when using checkpointing features or certain schedulers. To enable, set the train function arguments to be `func(config, checkpoint_dir=None)`.


Trial name,status,loc,alpha,beta
training_function_eddc8_00000,RUNNING,,0.001,2


Result for training_function_eddc8_00000:
  date: 2021-03-03_18-04-43
  done: false
  experiment_id: f8b79bbebb8d4ec7999bb8a7e7f8bf31
  hostname: 5719f135e538
  iterations_since_restore: 1
  mean_loss: 10.2
  neg_mean_loss: -10.2
  node_ip: 172.31.0.17
  pid: 120
  time_since_restore: 0.00029468536376953125
  time_this_iter_s: 0.00029468536376953125
  time_total_s: 0.00029468536376953125
  timestamp: 1614794683
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: eddc8_00000
  
Result for training_function_eddc8_00001:
  date: 2021-03-03_18-04-43
  done: false
  experiment_id: f499260dcba04c85adda37cc2bd34ada
  hostname: 5719f135e538
  iterations_since_restore: 1
  mean_loss: 10.1
  neg_mean_loss: -10.1
  node_ip: 172.31.0.17
  pid: 122
  time_since_restore: 0.00023245811462402344
  time_this_iter_s: 0.00023245811462402344
  time_total_s: 0.00023245811462402344
  timestamp: 1614794683
  timesteps_since_restore: 0
  training_iteration: 1
  trial_id: eddc8_00001
  
Result for

Trial name,status,loc,alpha,beta,loss,iter,total time (s),neg_mean_loss
training_function_eddc8_00000,TERMINATED,,0.001,2,10.191,10,0.061893,-10.191
training_function_eddc8_00001,TERMINATED,,0.01,1,10.0108,10,0.100214,-10.0108
training_function_eddc8_00002,TERMINATED,,0.1,1,9.27431,10,0.0914705,-9.27431


2021-03-03 18:04:43,650	INFO tune.py:450 -- Total run time: 1.18 seconds (0.88 seconds for the tuning loop).


Best config  {'alpha': 0.1, 'beta': 1}
